<a href="https://colab.research.google.com/github/mokraneimed/TensorFlow-projects/blob/main/Houari_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random

In [ ]:
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense
from tensorflow.keras.models import Sequential

In [ ]:
with open('content.json') as content:
  data1 = json.load(content)
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [ ]:
from sklearn import utils

In [ ]:
from os import replace
inputs, tags = utils.shuffle(inputs, tags)

In [ ]:
tokenizer = Tokenizer(num_words = 2000 , oov_token = '<UKN>')
tokenizer.fit_on_texts(inputs)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def get_sequences (tokenizer , tweets):
    sequences = tokenizer.texts_to_sequences(tweets)
    padded = pad_sequences(sequences , truncating='post',padding='post',maxlen=15)
    return padded

In [ ]:
padded_train = get_sequences(tokenizer,inputs)

In [ ]:
inputs[22]

" what's the basic requirements"

In [ ]:
padded_train[22]

array([78,  4, 36, 31,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
      dtype=int32)

In [ ]:
classes = set(tags)

In [ ]:
classes

{'goodbye',
 'greeting',
 'howami',
 'jokes',
 'prerequisites',
 'whereareyou',
 'whereisthecenter',
 'whoareyou'}

In [ ]:
class_to_index = dict((c,i) for i,c in enumerate(classes))
index_to_class = dict((c,i) for c,i in enumerate(classes) )

In [ ]:
index_to_class

{0: 'prerequisites',
 1: 'greeting',
 2: 'howami',
 3: 'whereisthecenter',
 4: 'jokes',
 5: 'whereareyou',
 6: 'goodbye',
 7: 'whoareyou'}

In [ ]:
class_to_index

{'prerequisites': 0,
 'greeting': 1,
 'howami': 2,
 'whereisthecenter': 3,
 'jokes': 4,
 'whereareyou': 5,
 'goodbye': 6,
 'whoareyou': 7}

In [ ]:
names_to_ids = lambda labels:([class_to_index.get(x) for x in labels])

In [ ]:
train_labels=names_to_ids(tags)

In [ ]:
train_labels

In [ ]:
len(padded_train)

81

In [ ]:
padded_val = padded_train[0:16]
val_labels = train_labels[0:16]

In [ ]:
len(val_labels)

16

In [ ]:
vocabulary = len(tokenizer.word_index)

In [ ]:
vocabulary

104

In [ ]:
padded_train = np.array(padded_train)
train_labels = np.array(train_labels)
padded_val = np.array(padded_val)
val_labels = np.array(val_labels)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(105,16,input_length= 15),
    tf.keras.layers.LSTM(10,return_sequences=True),
    tf.keras.layers.LSTM(10),
    tf.keras.layers.Dense(8,activation='softmax')
])
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
) 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_accuracy',mode='min',verbose=1,patience=2)

In [ ]:
h = model.fit(
    padded_train,train_labels,

    epochs=200,
)

Epoch 1/200
3/3 [==============================] - 4s 16ms/step - loss: 2.0795 - accuracy: 0.1358
Epoch 2/200
3/3 [==============================] - 0s 14ms/step - loss: 2.0761 - accuracy: 0.2099
Epoch 3/200
3/3 [==============================] - 0s 15ms/step - loss: 2.0738 - accuracy: 0.1358
Epoch 4/200
3/3 [==============================] - 0s 15ms/step - loss: 2.0713 - accuracy: 0.1235
Epoch 5/200
3/3 [==============================] - 0s 16ms/step - loss: 2.0686 - accuracy: 0.1235
Epoch 6/200
3/3 [==============================] - 0s 15ms/step - loss: 2.0658 - accuracy: 0.1235
Epoch 7/200
3/3 [==============================] - 0s 15ms/step - loss: 2.0631 - accuracy: 0.1358
Epoch 8/200
3/3 [==============================] - 0s 15ms/step - loss: 2.0600 - accuracy: 0.1358
Epoch 9/200
3/3 [==============================] - 0s 16ms/step - loss: 2.0566 - accuracy: 0.1358
Epoch 10/200
3/3 [==============================] - 0s 20ms/step - loss: 2.0528 - accuracy: 0.2222
Epoch 11/200
3/3 [=

In [ ]:
def respond(sentence):
  koko = []
  koko.append(sentence)
  seq = get_sequences (tokenizer , koko)
  preds =  model.predict(seq)[0]
  n = np.argmax(preds)
  response = random.choice(responses[index_to_class[n]])
  return response

In [ ]:
question = input()
print('Me : ',question)
print('Houari : ',respond(question))

how do i join the pirates
Me :  how do i join the pirates
1/1 [==============================] - 0s 18ms/step
Houari :  Yeah , am fine


In [ ]:
while True:
  question = input()
  print('Me : ',question)
  print('Houari : ',respond(question))

Me :  hi there
1/1 [==============================] - 0s 18ms/step
Houari :  Hey yo, do you need help ?
Me :  how do i join the pirates
1/1 [==============================] - 0s 21ms/step
Houari :  Yeah Fine, nothing much going on in my life
Me :  how do i join the pirates
1/1 [==============================] - 0s 19ms/step
Houari :  Things are Great
Me :  i want to join the pirates
1/1 [==============================] - 0s 18ms/step
Houari :  swordsman, a medical reindeer, taekwando Cook, master navigator, simple minded monkey, highly intelligent strategist, god usop. you gotta be one of these things to pass the requirement and join the pirates
Me :  alright , who are you ?
1/1 [==============================] - 0s 19ms/step
Houari :  galek jaja jabet lbac wch dart ? o9o9
Me :  what !!
1/1 [==============================] - 0s 23ms/step
Houari :  galek whd esmo m3amer 6al mn balcon sa7
Me :  how are you doing ?
1/1 [==============================] - 0s 21ms/step
Houari :  Yeah , am fi